# VIGA PREPRO
#### *Verificação de vigas biapoiadas, pré-fabricas, protendidas, com cabos retilíneos e protensão limitada*
<br>

Autores:
- Prof. Wanderlei Malaquias. Pereira Junior;
- Profa. Sylvia Regina Mesquisa de Almeida;
- Eng. Civil Gustavo Gonçalves Costa;
- Eng. Civil Matheus Henrique Morato de Moraes;
- Eng. Civil Geraldo Magela Gonçalves Filho

<p align="justify">Antes de iniciar o exemplo execute a célula de instalação do programa. Caso deseje rodar mais de um vez não é necessário instalar a biblioteca novamente.</p>

_Produzido em linguagem Python_  
_Licença MIT_

In [1]:
# Célula de instalação (**Não é necessário executar novamente quando precisar mudar os dados**)
import numpy as np
from VIGA_PREPRO import *
from PERDAS import *

<p align="justify">Prezado usuário digite os dados de entrada da sua viga biapoiada. <b>Antes de iniciar</b> o processo <b>verifique as unidades de entrada</b> na documentação.</p>

<a href="https://wmpjrufg.github.io/VIGA-PREPRO/" target="_blank">Clique aqui para acessar a documentação</a>

In [2]:
# Geometria da viga
H = 0.70
B_FS = 0.30
B_FI = 0.30
B_W = 0.11
H_S = 0.09
H_I = 0.09
H_SI = 0.07
H_II = 0.07
L = 20
PHI_L = 12.5 / 1E3
PHI_E = 10.0 / 1E3
COB = 0.025 
# Características do aço
E_SCP = 195E6
GAMMA_S = 78 
TIPO_PROT = 'PRE'
TIPO_ACO = 'RB'
F_PK = 1864210.526
F_YK = 1676573.427
LAMBA_SIG = 0.85
# Características do concreto
GAMMA_C = 25
F_CK = 40 * 1E3
CIMENTO = 'CP5'
AGREGADO = 'GRA'
PHI_FLUENCIA0 = 3.70
PHI_FLUENCIA1 = 1.70
# Carregamentos e coeficientes parciais
G_2K = 1.55 
G_3K = 0.70
Q_1K = 1.5
Q_2K = 0.0
PSI_1 = 0.40
PSI_2 = 0.30
GAMMA_F = 1.40
GAMMA_S = 1.15
# Tempos do concreto
TEMPO_CONC = [1, 50 * 365]
# Perdas estimadas em percentual
PERDA_INICIAL = 7.87
PERDA_TEMPO = 23.81 - 7.87

# Aderência das cordoalhas
ETA_1 = 1.2
ETA_2 = 1.0

## A célula abaixo verifica:
- Propriedades geométricas da seção retangular;
- Propriedades do material na idade da aplicação da protensão e infinito (50 anos de uso);
- Tensão inicial nos cabos.



In [3]:
# Propriedades da seção
[A_C, I_C, Y_SUP, Y_INF, W_SUP, W_INF] = PROP_GEOMETRICA_RET(B_W,H)  
E_P = Y_INF - COB - (PHI_L / 2) - PHI_E
print('Propriedades da seção transversal')
print('A_C:   ', '%+10.5e' % A_C, 'm²')
print('I_C:   ', '%+10.5e' % I_C, 'm^4')
print('Y_SUP: ', '%+10.5e' % Y_SUP, 'm')
print('Y_INF: ', '%+10.5e' % Y_INF, 'm')
print('W_SUP: ', '%+10.5e' % W_SUP, 'm³')
print('W_INF: ', '%+10.5e' % W_INF, 'm³')
print('E_P:   ', '%+10.5e' % E_P, 'm')
print('\n')
[A_C, I_C, Y_SUP, Y_INF, W_SUP, W_INF] = PROP_GEOMETRICA_I(H, B_FS, B_FI, B_W, H_S, H_I, H_SI, H_II)
E_P = Y_INF - COB - (PHI_L / 2) - PHI_E
G_1K = A_C * GAMMA_C 
print('Propriedades da seção transversal AQUI I')
print('A_C:   ', '%+10.5e' % A_C, 'm²')
print('I_C:   ', '%+10.5e' % I_C, 'm^4')
print('Y_SUP: ', '%+10.5e' % Y_SUP, 'm')
print('Y_INF: ', '%+10.5e' % Y_INF, 'm')
print('W_SUP: ', '%+10.5e' % W_SUP, 'm³')
print('W_INF: ', '%+10.5e' % W_INF, 'm³')
print('E_P:   ', '%+10.5e' % E_P, 'm')
print('\n')

# Propriedades do material em todos as etapas
F_CKJ = []; F_CTMJ = []; F_CTKINFJ = []; F_CTKSUPJ = []; E_CIJ = []; E_CSJ = [];
print('Propriedades do material')
for I_COUNT in range(len(TEMPO_CONC)):
    TEMPO = TEMPO_CONC[I_COUNT]
    [F_CKJJ, F_CTM, F_CTKINF, F_CTKSUP, E_CI, E_CS] = PROP_MATERIAL(F_CK, TEMPO, CIMENTO, AGREGADO)
    F_CKJ.append(F_CKJJ); F_CTMJ.append(F_CTM); F_CTKINFJ.append(F_CTKINF)
    F_CTKSUPJ.append(F_CTKSUP); E_CIJ.append(E_CI), E_CSJ.append(E_CS) 
    print('Idade:    ', TEMPO, 'dias')
    print('F_CKJ:    ', '%+10.5e' % F_CKJJ, 'kN/m²')
    print('F_CTM:    ', '%+10.5e' % F_CTM, 'kN/m²')
    print('F_CTKINF: ', '%+10.5e' % F_CTKINF, 'kN/m²')
    print('F_CTKSUP: ', '%+10.5e' % F_CTKSUP, 'kN/m²')
    print('E_CI:     ', '%+10.5e' % E_CI, 'kN/m²')
    print('E_CS:     ', '%+10.5e' % E_CS, 'kN/m²')
    print('\n')

# Tensão inicial
SIGMA_PI0 = TENSAO_INICIAL(TIPO_PROT, TIPO_ACO, F_PK, F_YK)
SIGMA_PI0 *= LAMBA_SIG
print('Tensão inicial na armadura')
print('SIGMA_PI0: ', '%+10.5e' % SIGMA_PI0, 'kN/m²')

Propriedades da seção transversal
A_C:    +7.70000e-02 m²
I_C:    +3.14417e-03 m^4
Y_SUP:  +3.50000e-01 m
Y_INF:  +3.50000e-01 m
W_SUP:  +8.98333e-03 m³
W_INF:  +8.98333e-03 m³
E_P:    +3.08750e-01 m


Propriedades da seção transversal AQUI I
A_C:    +1.24500e-01 m²
I_C:    +7.09727e-03 m^4
Y_SUP:  +3.50000e-01 m
Y_INF:  +3.50000e-01 m
W_SUP:  +2.02779e-02 m³
W_INF:  +2.02779e-02 m³
E_P:    +3.08750e-01 m


Propriedades do material
Idade:     1 dias
F_CKJ:     +2.10000e+04 kN/m²
F_CTM:     +2.28350e+03 kN/m²
F_CTKINF:  +1.59845e+03 kN/m²
F_CTKSUP:  +2.96855e+03 kN/m²
E_CI:      +2.56624e+07 kN/m²
E_CS:      +2.30962e+07 kN/m²


Idade:     18250 dias
F_CKJ:     +4.00000e+04 kN/m²
F_CTM:     +3.50882e+03 kN/m²
F_CTKINF:  +2.45617e+03 kN/m²
F_CTKSUP:  +4.56147e+03 kN/m²
E_CI:      +3.54175e+07 kN/m²
E_CS:      +3.18758e+07 kN/m²


Tensão inicial na armadura
SIGMA_PI0:  +1.21132e+06 kN/m²


## A célula abaixo verifica:
- O comprimento de transferência da peça.

In [4]:
# Determinação do comprimento de transferência
SIGMA_PIINI = SIGMA_PI0 - SIGMA_PI0 * (PERDA_INICIAL / 100)
PERDA_TOTAL = PERDA_INICIAL + PERDA_TEMPO
SIGMA_PIINF = SIGMA_PI0 - SIGMA_PI0 * (PERDA_TOTAL / 100)
L_PINI = COMPRIMENTO_TRANSFERENCIA(PHI_L, F_YK, F_CTKINFJ[0], ETA_1, ETA_2, SIGMA_PIINI, H)
L_PINF = COMPRIMENTO_TRANSFERENCIA(PHI_L, F_YK, F_CTKINFJ[1], ETA_1, ETA_2, SIGMA_PIINF, H)
L_P = np.mean([L_PINI, L_PINF])
print('Comprimento de transferência considerando as perdas iniciais')
print('L_P: ', '%+10.5e' % L_P, 'm')
print('\n')

Comprimento de transferência considerando as perdas iniciais
L_P:  +1.02168e+00 m




## A célula abaixo verifica:
- Esforços de flexão meio do vão e apoios (no comp. de transferência);
- Cisalhamento nos apoios.

In [5]:
# Momento máximo (meio do vão), momento apoios (no comp. de transferência) e cortante máximo (apoios)
[M_MVG1K, M_APG1K, V_APG1K] = ESFORCOS(G_1K, L, L_P)
[M_MVG2K, M_APG2K, V_APG2K] = ESFORCOS(G_2K, L, L_P)
[M_MVG3K, M_APG3K, V_APG3K] = ESFORCOS(G_3K, L, L_P)
[M_MVQ1K, M_APQ1K, V_APQ1K] = ESFORCOS(Q_1K, L, L_P)
print('Esforços devido aos carregametos G e Q (M = kN.m, V = kN)')
print('MV = Meio do vão, AP = Apoio')
print('M_G1MV:  ', '%+10.5e' % M_MVG1K, '   M_G1AP:  ', '%+10.5e' % M_APG1K,  '   V_G1AP:  ', '%+10.5e' % V_APG1K)
print('M_G2MV:  ', '%+10.5e' % M_MVG2K, '   M_G2AP:  ', '%+10.5e' % M_APG2K,  '   V_G2AP:  ', '%+10.5e' % V_APG2K)
print('M_G3MV:  ', '%+10.5e' % M_MVG3K, '   M_G3AP:  ', '%+10.5e' % M_APG3K,  '   V_G2AP:  ', '%+10.5e' % V_APG3K)
print('M_Q1MV:  ', '%+10.5e' % M_MVQ1K, '   M_Q1AP:  ', '%+10.5e' % M_APQ1K,  '   V_Q1AP:  ', '%+10.5e' % V_APQ1K)
print('\n')

Esforços devido aos carregametos G e Q (M = kN.m, V = kN)
MV = Meio do vão, AP = Apoio
M_G1MV:   +1.55625e+02    M_G1AP:   +3.09876e+01    V_G1AP:   +3.11250e+01
M_G2MV:   +7.75000e+01    M_G2AP:   +1.54316e+01    V_G2AP:   +1.55000e+01
M_G3MV:   +3.50000e+01    M_G3AP:   +6.96911e+00    V_G2AP:   +7.00000e+00
M_Q1MV:   +7.50000e+01    M_Q1AP:   +1.49338e+01    V_Q1AP:   +1.50000e+01




## A célula abaixo verifica:
- Determinação da armadura A_SCP longitudinal que respeite a equação de Estado Limite de Serviço (ELS-F) no tempo infinito.

In [6]:
# Avaliação no apoio
A_SCPINICIAL0 = ARMADURA_ASCP_ELS(A_C, W_INF, E_P, PSI_1, PSI_2, M_APG1K, M_APG2K, M_APG3K, M_APQ1K, SIGMA_PIINF, F_CTKINFJ[1])

# Avaliação no meio do vão
A_SCPINICIAL1 = ARMADURA_ASCP_ELS(A_C, W_INF, E_P, PSI_1, PSI_2, M_MVG1K, M_MVG2K, M_MVG3K, M_MVQ1K, SIGMA_PIINF, F_CTKINFJ[1])

# Seleção do maior valor (pior situação)
A_SCP = max(A_SCPINICIAL0, A_SCPINICIAL1)
print('Sugestão de armadura ')
print('A_SCPINICIAL >=  ', '%+10.5e' % A_SCPINICIAL0, 'm²', '  Sugestão de armadura respeitando as condições nos apoios')
print('A_SCPINICIAL >=  ', '%+10.5e' % A_SCPINICIAL1, 'm²', '  Sugestão de armadura respeitando as condições no meio do vão')
print('\n')
print('A_SCP        ==  ', '%+10.5e' % A_SCP, 'm²', '  Armadura inicial escolhida')

Sugestão de armadura 
A_SCPINICIAL >=   +1.32950e-04 m²   Sugestão de armadura respeitando as condições nos apoios
A_SCPINICIAL >=   +6.67695e-04 m²   Sugestão de armadura respeitando as condições no meio do vão


A_SCP        ==   +6.67695e-04 m²   Armadura inicial escolhida


## A célula abaixo verifica:
- Verifica as tensões normais no ato da protensão considerando um Estado Limite Último (ELU).

In [7]:
print('Verificações ELU no ato da protensão')
# Carga inicial de protensão após definição da armadura
SIGMA_PI1 = SIGMA_PIINI
P_I1 = SIGMA_PI1 * A_SCP
print('Parâmetros da protensão')
print('SIGMA_PI1:     ' , '%+10.5e' % SIGMA_PI1, 'kN/m²')
print('P_I1:          ' , '%+10.5e' % P_I1, 'kN')
print('\n')

# Valores máximos das tensões (tração e compressão)
SIGMA_TRACMAX = 1.20 * F_CTMJ[0]
SIGMA_COMPMAX = 0.70 * F_CKJ[0]
print('Valores máximos das tensões ')
print('SIGMA_TRACMAX: ' , '%+10.5e' % SIGMA_TRACMAX, 'kN/m²')
print('SIGMA_COMPMAX: ' , '%+10.5e' % SIGMA_COMPMAX, 'kN/m²')
print('\n')

# Verificação tensões normais no ato da protensão no apoio
G = []
[SIGMA_INF, SIGMA_SUP] = TENSOES_NORMAIS(P_I1, A_C, E_P, W_INF, W_SUP, 1, 1, 0, 0, 0, 0, 0, M_APG1K, 0, 0, 0, 0)
[G_0, G_1] = VERIFICA_TENSAO_NORMAL_ATO_PROTENSÃO(SIGMA_INF, SIGMA_SUP, SIGMA_TRACMAX, SIGMA_COMPMAX)
G.append(G_0); G.append(G_1)
print('Verificação das tensões normais nos bordos no apoio ')
print('Bordo inferior:', '%+10.5e' % SIGMA_INF, 'kN/m²', '// Eq. Estado Limite:', '%+10.5e' % G_0)
print('Bordo superior:', '%+10.5e' % SIGMA_SUP, 'kN/m²', '// Eq. Estado Limite:', '%+10.5e' % G_1)
print('\n')

# Verificação tensões normais no ato da protensão no meio do vão
[SIGMA_INF, SIGMA_SUP] = TENSOES_NORMAIS(P_I1, A_C, E_P, W_INF, W_SUP, 1, 1, 0, 0, 0, 0, 0, M_MVG1K, 0, 0, 0, 0)
[G_2, G_3] = VERIFICA_TENSAO_NORMAL_ATO_PROTENSÃO(SIGMA_INF, SIGMA_SUP, SIGMA_TRACMAX, SIGMA_COMPMAX)
G.append(G_2); G.append(G_3)
print('Verificação das tensões normais nos bordos no meio do vão ')
print('Bordo inferior:', '%+10.5e' % SIGMA_INF, 'kN/m²', '// Eq. Estado Limite:', '%+10.5e' % G_2)
print('Bordo superior:', '%+10.5e' % SIGMA_SUP, 'kN/m²', '// Eq. Estado Limite:', '%+10.5e' % G_3)
print('\n')

Verificações ELU no ato da protensão
Parâmetros da protensão
SIGMA_PI1:      +1.11599e+06 kN/m²
P_I1:           +7.45142e+02 kN


Valores máximos das tensões 
SIGMA_TRACMAX:  +2.74020e+03 kN/m²
SIGMA_COMPMAX:  +1.47000e+04 kN/m²


Verificação das tensões normais nos bordos no apoio 
Bordo inferior: +1.58024e+04 kN/m² // Eq. Estado Limite: +7.49938e-02
Bordo superior: -3.83225e+03 kN/m² // Eq. Estado Limite: +3.98530e-01


Verificação das tensões normais nos bordos no meio do vão 
Bordo inferior: +9.65595e+03 kN/m² // Eq. Estado Limite: -3.43132e-01
Bordo superior: +2.31421e+03 kN/m² // Eq. Estado Limite: -8.42571e-01




## A célula abaixo verifica:
- Determina a armadura longitudinal necessária A_SCPNEC para a peça.

In [8]:
print('Verificação da armadura necessária')
# Momento de cálculo
M_SD = (M_MVG1K + M_MVG2K + M_MVG3K + M_MVQ1K) * GAMMA_F
print('Momento de cálculo')
print('M_SD: ', '%+10.5e' % M_SD, 'kN.m')
print('\n')

# Altura útil
D = Y_SUP + E_P
print('Altura útil')
print('D: ',   '%+10.5e' % D, 'm')
print('\n')

# Propriedades do aço
F_PD = F_PK / GAMMA_S
F_YD = F_YK / GAMMA_S
EPSILON_Y = F_YD / E_SCP
print('Propriedades do aço')
print('F_PD:      ',   '%+10.5e' % F_PD, 'kN/m²')
print('F_YD:      ',   '%+10.5e' % F_YD, 'kN/m²')
print('EPSILON_Y: ',   '%+10.5e' % EPSILON_Y)
print('\n')

# Armadura necessária
X, EPSILON_S, EPSILON_C, Z, A_SCPNEC = AREA_ACO_FNS_RETANGULAR_SIMPLES('CP', M_SD, F_CK, B_W, D, E_SCP, SIGMA_PIINF, 35/1000, EPSILON_Y, F_PD, F_YD)
G_4 = VERIFICA_ARMADURA_FLEXAO(A_SCP, A_SCPNEC)
G.append(G_4)
print('Armadura necessária')
print('X:         ',   '%+10.5e' % X, 'm')
print('EPSILON_S: ',   '%+10.5e' % EPSILON_S)
print('EPSILON_C: ',   '%+10.5e' % EPSILON_C)
print('Z:         ',   '%+10.5e' % Z, 'm')
print('A_SCPNEC:  ',   '%+10.5e' % A_SCPNEC, 'm²', '// Eq. Estado Limite:', '%+10.5e' % G_4)
print('\n')

Verificação da armadura necessária
Momento de cálculo
M_SD:  +4.80375e+02 kN.m


Altura útil
D:  +6.58750e-01 m


Propriedades do aço
F_PD:       +1.62105e+06 kN/m²
F_YD:       +1.45789e+06 kN/m²
EPSILON_Y:  +7.47636e-03


Armadura necessária
X:          +4.82683e-01 m
EPSILON_S:  +9.35458e-04
EPSILON_C:  +3.50000e-03
Z:          +4.65677e-01 m
A_SCPNEC:   +9.33269e-04 m² // Eq. Estado Limite: +3.97749e-01




## A célula abaixo verifica:
- Determina a resistência da biela de compressão;
- Determina a armadura transversal A_SW necessária para a peça.

In [9]:
# Verificação da biela comprimida
V_SD = (V_APG1K + V_APG2K + V_APG3K + V_APQ1K) * 1.40
V_RD2 = RESISTENCIA_BIELA_COMPRIMIDA(F_CKJ[1], B_W, D)
G_5 = VERIFICA_BIELA_COMPRIMIDA(V_SD, V_RD2)
G.append(G_5)
print('Verificação da biela de compressão')
print('Esforço cortante solicitante')
print('V_SD:         ',   '%+10.5e' % V_SD, 'kN')
print('\n')
print('Resistência da biela de compressão')
print('V_RD2:        ',   '%+10.5e' % V_RD2, 'kN', '// Eq. Estado Limite:', '%+10.5e' % G_5)
print('\n')

# Cálculo da armadura necessária
P_IINF = SIGMA_PIINF * A_SCP
V_C, V_SW, A_SW = AREA_ACO_TRANSVERSAL_MODELO_I(0, P_IINF, V_SD, F_CTKINFJ[1], B_W, D, 'CP', W_INF, A_C, E_P, M_SD, F_CTMJ[1], 500E3)
print('Resistência do concreto ao cisalhamento')
print('V_C:          ',   '%+10.5e' % V_C, 'kN')
print('\n')
print('Carga de cisalhamento absorvida pelo estribo')
print('V_SW:         ',   '%+10.5e' % V_SW, 'kN')
print('\n')
print('Armadura para o cisalhamento')
print('A_SW:         ',   '%+10.5e' % A_SW, 'm²/m')
print('\n')

Verificação da biela de compressão
Esforço cortante solicitante
V_SD:          +9.60750e+01 kN


Resistência da biela de compressão
V_RD2:         +4.69557e+02 kN // Eq. Estado Limite: -7.95392e-01


Resistência do concreto ao cisalhamento
V_C:           +1.17810e+02 kN


Carga de cisalhamento absorvida pelo estribo
V_SW:          +0.00000e+00 kN


Armadura para o cisalhamento
A_SW:          +1.54388e-04 m²/m




## A célula abaixo verifica:
- Propriedades no Estádios I e II;
- Momento de fissuração;
- Flecha.

In [10]:
print("Verificação da flecha")
# Determinação das propriedades no Estádio I
ALPHA_MOD = E_SCP / E_CSJ[1]
A_CI, X_I, I_I = GEOMETRIC_PROPERTIES_STATE_I(H, B_W, B_W, 0, A_SCP, ALPHA_MOD, D)
M_R = M_R_BENDING_MOMENT(2, F_CTMJ[1], H, X_I, I_I, P_IINF, A_CI, W_INF, E_P)

# Determinação das propriedades no Estádio II
X_II, I_II = GEOMETRIC_PROPERTIES_STATE_II(H, B_W, B_W, 0, A_SCP, 0, ALPHA_MOD, D, 0)
print("Propriedades no Estádio I")
print('Área da seção (Ac)               = ', A_CI, 'm²')
print('Linha Neutra (xi)                = ', X_I, 'm')
print('Inércia Estádio I (Ii)           = ', I_I, 'm^4')
print('Momento resistente (Mr serviço)  = ', M_R, 'kN.m')
print('\n')
print("Propriedades no Estádio II")
print('Linha Neutra Estádio II (xii)    = ', X_II, 'm')
print('Inércia Estádio II (Iii)         = ', I_II, 'm^4')
print('\n')

# Momento atuante
M_SER = (M_MVG1K + M_MVG2K + M_MVG3K) + (M_MVQ1K) * PSI_2
print("Verificação do Estádio")
print('Momento atuante (Mat serviço)    = ', M_SER, 'kN.m')

# Inércia e rigidez equivalente da peça
if M_SER > M_R:
    I_M = BRANSON_INERTIA(M_R, M_SER, I_I, I_II)
    I_CNOVO = I_M
    print('Peça deverá ser verificada no Estádio II')
else:
    I_CNOVO  = I_I
    print('Peça deverá ser verificada no Estádio I')
print('\n')

EI0 = E_CSJ[0] * I_CNOVO
EI1 = E_CSJ[1] * I_CNOVO
print("Avaliação da inércia equivalente")
print("Inércia equivalente               =", I_CNOVO, 'm^4')
print("Rigidez equivalente na protensão  =", EI0, 'kN.m²')
print("Rigidez equivalente no serviço    =", EI1, 'kN.m²')

# Cálculo das flechas totais
M_PI = P_I1 * E_P
A_PI = -(M_PI * (L ** 2)) / (8 * EI0)
A_PI *= (PHI_FLUENCIA0 + 1)
A_G1 = (5 * G_1K * (L ** 4) / (384 * EI0))
A_G1 *= (PHI_FLUENCIA0 + 1)
G_SER = G_1K + G_2K + G_3K + PSI_2 * Q_1K
A_G23 = (5 * (G_2K + G_3K) * (L ** 4) / (384 * EI1))
A_G23 *= (PHI_FLUENCIA1 + 1)
A_Q1 = (5 * (PSI_2 * Q_1K) * (L ** 4) / (384 * EI1))
A_GT = A_G1 + A_G23 + A_Q1
A_FABRICA = A_PI + A_G1
A_SERVICO = A_PI + A_GT

# Verificação flecha na fabricação
G_6 = VERIFICA_FLECHA(A_FABRICA, L / 1000)
G_7 = VERIFICA_FLECHA(A_SERVICO, L / 250)
print("Flecha total (Direta + Fluência)")
print("Carga de serviço                = ", G_SER, 'kN/m')
print("Flecha total na fabricação      = ", A_FABRICA, 'm', '// Eq. Estado Limite:', G_6)
print("Flecha total no serviço         = ", A_SERVICO, 'm', '// Eq. Estado Limite:', G_7)

Verificação da flecha
Propriedades no Estádio I
Área da seção (Ac)               =  0.08041692767966802 m²
Linha Neutra (xi)                =  0.36311883519474764 m
Inércia Estádio I (Ii)           =  0.003456050574877797 m^4
Momento resistente (Mr serviço)  =  399.63955983460124 kN.m


Propriedades no Estádio II
Linha Neutra Estádio II (xii)    =  0.1871470043255724 m
Inércia Estádio II (Iii)         =  0.001148795001155702 m^4


Verificação do Estádio
Momento atuante (Mat serviço)    =  290.625 kN.m
Peça deverá ser verificada no Estádio I


Avaliação da inércia equivalente
Inércia equivalente               = 0.003456050574877797 m^4
Rigidez equivalente na protensão  = 79821.5713594647 kN.m²
Rigidez equivalente no serviço    = 110164.23457550928 kN.m²
Flecha total (Direta + Fluência)
Carga de serviço                =  5.812500000000001 kN/m
Flecha total na fabricação      =  -0.29551135439871484 m // Eq. Estado Limite: -15.775567719935742
Flecha total no serviço         =  -0.17211604